In [14]:
import fitz

In [54]:
# PDF to Image, use OCR to detect bounding boxes of text
from pdf2image import convert_from_path
import easyocr
import numpy as np
pdf_path = 'Structured_note_termsheet/cct-linked-structured-note.pdf'
images = convert_from_path(pdf_path)
reader = easyocr.Reader(['en'])
bounds = reader.readtext(np.array(images[0]), min_size=10, slope_ths=0.5, ycenter_ths=0.7, height_ths=1.2, width_ths=0.85, decoder='beamsearch', beamWidth=10)

In [56]:
# export the image to a file
images[0].save('output.jpg', 'JPEG')

In [36]:
# Get the coordinates bounding box of the text in the PDF
def search_text_and_get_coordinates(pdf_path, search_text):
    document = fitz.open(pdf_path)
    
    start_text_coordinates = (float('inf'), float('inf'))
    end_text_coordinates = (float('-inf'), float('-inf'))
    
    for page_number in range(len(document)):
        page = document.load_page(page_number)
        
        text_instances = page.search_for(search_text)
        
        for inst in text_instances:
            x0, y0, x1, y1 = inst
            if start_text_coordinates is None:
                start_text_coordinates = (x0, y0)
            else:
              # choose the smallest x0 and y0
              start_text_coordinates = (min(start_text_coordinates[0], x0), min(start_text_coordinates[1], y0))
        
            if end_text_coordinates is None:
                end_text_coordinates = (x1, y1)
            else:
              # choose the largest x1 and y1
              end_text_coordinates = (max(end_text_coordinates[0], x1), max(end_text_coordinates[1], y1))
              
    return start_text_coordinates, end_text_coordinates

pdf_path = 'Structured_note_termsheet/cct-linked-structured-note.pdf'

search_text = 'Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.'

search_text_and_get_coordinates(pdf_path, search_text)

((86.30400085449219, 280.5127868652344), (526.9910888671875, 340.22802734375))

In [42]:
data_json = {
  "training_data": [
    {
      "text": "Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.",
      "start_text_coordinates": {
        "x": 86.30400085449219,
        "y": 280.5127868652344
      },
      "end_text_coordinates": {
        "x": 526.9910888671875,
        "y": 340.22802734375
      },
      "expected_output": "<StatusOfNotes><Description>The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.</Description></StatusOfNotes>"
    }
  ]
}

# tabulur data scaling 
# key scheching
# page breaks
